<a href="https://colab.research.google.com/github/diegogrosmann/CSP/blob/main/notebooks/otimizacao_blfga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Otimização do Algoritmo Genético BLF-GA para o Closest String Problem (CSP)

Este notebook tem como objetivo analisar, testar e otimizar o desempenho do algoritmo BLF-GA aplicado ao CSP, seguindo um processo iterativo de experimentação e melhoria.

# 1. Inicializar as configurações básicas

In [1]:
import os

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

REPO_URL = "https://github.com/diegogrosmann/CSP.git"
REPO_DIR = "CSP"

if is_colab():
    # Só executa esse bloco no Google Colab
    if os.path.isdir(REPO_DIR):
        print(f"Pasta '{REPO_DIR}' já existe. Atualizando (git pull)...")
        os.chdir(REPO_DIR)
        !git reset --hard
        !git pull
    else:
        print(f"Pasta '{REPO_DIR}' não existe. Clonando repositório...")
        !git clone {REPO_URL}
        os.chdir(REPO_DIR)
    %cd notebooks
    print(f"Agora você está na pasta: {os.getcwd()}")
else:
    print("Você não está no Google Colab! Nada foi executado.")



Você não está no Google Colab! Nada foi executado.


In [2]:
# Corrigindo o PYTHONPATH para permitir imports relativos ao projeto
import sys, os
sys.path.insert(0, os.path.abspath('..'))

## 2. Parâmetros do Dataset e Algoritmo

Abaixo, você pode sobreescrever os parâmetros padrão tanto para o dataset sintético quanto para o algoritmo BLF-GA. Basta definir a opção desejada em cada bloco de parâmetros.

In [3]:
# Parâmetros do dataset sintético: sempre carrega padrão e sobrescreve se informado manualmente
from utils.config import SYNTHETIC_DEFAULTS

dataset_params = SYNTHETIC_DEFAULTS.copy()

# Para sobrescrever, basta definir manualmente abaixo (exemplo):
dataset_params["n"] = 80
dataset_params["L"] = 300
dataset_params["alphabet"] = "ACGT"
dataset_params["noise"] = 0.4
dataset_params["seed"] = 1

print("Parâmetros do dataset:")
print(f" - n: {dataset_params['n']}\n - L: {dataset_params['L']}\n - alphabet: {dataset_params['alphabet']}\n - noise: {dataset_params['noise']}\n - seed: {dataset_params['seed']}")

Parâmetros do dataset:
 - n: 80
 - L: 300
 - alphabet: ACGT
 - noise: 0.4
 - seed: 1


In [4]:
# Defina aqui os valores desejados para cada parâmetro (lista ou valor único)
blfga_param_grid = {
    'pop_size': [150, 200],                         # Tamanho da população
    'initial_blocks': [20, 40],                     # Número de blocos iniciais
    'min_block_len': [1],                           # Tamanho mínimo do bloco
    'cross_prob': [0.9],                            # Probabilidade de crossover
    'mut_prob': [0.5, 0.1],                         # Probabilidade de mutação
    'elite_rate': [0.05],                           # Taxa de elite
    'rediv_freq': [10],                             # Frequência de redivisão
    'max_gens': [400],                              # Número máximo de gerações
    'max_time': [1200.0],                           # Tempo máximo em segundos
    'seed': [1],                                    # Semente para reprodutibilidade
    'immigrant_freq': [10],                         # Gera imigrantes a cada X gerações
    'immigrant_ratio': [0.2],                       # Proporção de imigrantes
    'diversity_threshold': [0.4],                   # Limite para diversidade
    'mutation_adapt_N': [10],                       # N gerações para detectar convergência
    'mutation_adapt_factor': [2.0, 3.0],            # Fator de aumento temporário da mutação
    'mutation_adapt_duration': [5],                 # Duração do aumento da mutação
    'mutation_type': ['multi', 'transposition'],    # Tipo de mutação: multi, inversion, transposition
    'mutation_multi_n': [2, 3],                     # Número de posições para mutação multi
    'tournament_k': [2, 3],                         # Parâmetro externo do torneio
    'crossover_type': ['one_point'],                # Tipo de crossover: one_point, uniform, blend_blocks
    'niching': [False, True],                       # Ativa niching
    'niching_radius': [3],                          # Raio de nicho
    'refinement_type': ['greedy'],                  # Tipo de refinamento: greedy, swap, insertion, 2opt
    'refine_elites': ['best'],                      # Elites a refinar: all, best
    'refine_iter_limit': [100],                     # Limite de iterações por refinamento
    'restart_patience': [20],                       # Gerações sem melhoria para restart
    'restart_ratio': [0.3],                         # Proporção da população a reiniciar
    'disable_elitism_gens': [0],                    # Gerações sem elitismo
}

In [5]:
# Parâmetros do algoritmo BLF-GA: permite grid de configurações
from algorithms.blf_ga.config import BLF_GA_DEFAULTS
from itertools import product

# Preenche com o padrão se não especificado
param_names = list(BLF_GA_DEFAULTS.keys())
param_values = [blfga_param_grid.get(k, [BLF_GA_DEFAULTS[k]]) if not isinstance(blfga_param_grid.get(k, BLF_GA_DEFAULTS[k]), list) else blfga_param_grid.get(k, [BLF_GA_DEFAULTS[k]]) for k in param_names]

# Gera todas as combinações
blfga_experimentos = list(product(*param_values))

print(f"Total de configurações BLF-GA: {len(blfga_experimentos)}")
for i, valores in enumerate(blfga_experimentos):
    print(f"Configuração {i+1}:")
    for k, v in zip(param_names, valores):
        print(f" - {k}: {v}")

Total de configurações BLF-GA: 256
Configuração 1:
 - pop_size: 150
 - initial_blocks: 20
 - min_block_len: 1
 - cross_prob: 0.9
 - mut_prob: 0.5
 - elite_rate: 0.05
 - rediv_freq: 10
 - max_gens: 400
 - max_time: 1200.0
 - seed: 1
 - immigrant_freq: 10
 - immigrant_ratio: 0.2
 - diversity_threshold: 0.4
 - mutation_adapt_N: 10
 - mutation_adapt_factor: 2.0
 - mutation_adapt_duration: 5
 - mutation_type: multi
 - mutation_multi_n: 2
 - tournament_k: 2
 - crossover_type: one_point
 - niching: False
 - niching_radius: 3
 - refinement_type: greedy
 - refine_elites: best
 - refine_iter_limit: 100
 - restart_patience: 20
 - restart_ratio: 0.3
 - disable_elitism_gens: 0
Configuração 2:
 - pop_size: 150
 - initial_blocks: 20
 - min_block_len: 1
 - cross_prob: 0.9
 - mut_prob: 0.5
 - elite_rate: 0.05
 - rediv_freq: 10
 - max_gens: 400
 - max_time: 1200.0
 - seed: 1
 - immigrant_freq: 10
 - immigrant_ratio: 0.2
 - diversity_threshold: 0.4
 - mutation_adapt_N: 10
 - mutation_adapt_factor: 2.0
 -

# 2.1 Geração do Dataset Sintético

Abaixo será gerado um dataset sintético de strings para o Closest String Problem (CSP), utilizando os parâmetros definidos acima (número de strings, comprimento, alfabeto, nível de ruído e semente).

O objetivo é criar um conjunto de dados controlado e reprodutível para testar e comparar o desempenho do algoritmo BLF-GA.

In [6]:
# Gerar dataset sintético com os parâmetros definidos (nova forma)
from datasets.dataset_synthetic import generate_dataset_with_params

strings, params_usados = generate_dataset_with_params(dataset_params)
print(f"Dataset gerado: n={len(strings)}, L={len(strings[0])}, |Σ|={len(params_usados['alphabet'])}")
print("Parâmetros usados:")
for k, v in params_usados.items():
    print(f" - {k}: {v}")

Parâmetros usados: n=80, L=300, alphabet='ACGT', noise=0.4, fully_random=False
Distância da string base: 138, Semente: 1                             
Dataset gerado: n=80, L=300, |Σ|=4
Parâmetros usados:
 - n: 80
 - L: 300
 - alphabet: ACGT
 - fully_random: False
 - base_string: ATTCCCGTAATCTACGATTAAGTCACAACCAAACCATGGATTACGGTCTGCGTTGGAATCAGGGCCGTGCCAAGTGCAGTTGTAGTGCCGTATTTGTGGCATGAGCCCGGGCAAAGTTTTCTGAAATAAGCAAGACGCCCACCAATGAGTAAAGAGGGATTGAGCGCGACTTCTCTGCCATATTGATTGGCCAGCAAGCCCTTAACTTCAGTTCTGCTAGAATATGTCCCTGTTAGAAATTTCGTCGAACTGTCCTTAGAATAATCAAAGATCTTCCCAGAATCGCCATTTAAGTGGGCG
 - noise: 0.4
 - seed: 1
 - distancia_string_base: 138


## 3. Reexecução e Comparação dos Resultados

Executando novamente o BLF-GA com outros parametros para analise.

In [7]:
# Função utilitária para executar o BLF-GA e retornar histórico
def executar_blfga_com_hist(strings, alphabet, params):
    import time
    from algorithms.blf_ga.algorithm import BLFGAAlgorithm
    alg = BLFGAAlgorithm(strings, alphabet, **params)
    t0 = time.time()
    center, dist, history = alg.run_with_history()
    t1 = time.time()
    return center, dist, t1-t0, history


In [8]:
# Experimentos automáticos: variação de parâmetros do BLF-GA usando o novo grid
import pandas as pd
import matplotlib.pyplot as plt

resultados = []

for i, valores in enumerate(blfga_experimentos):
    params = {k: v for k, v in zip(param_names, valores)}
    # Se quiser sobrescrever só alguns parâmetros, pode atualizar aqui
    center, dist, tempo, history = executar_blfga_com_hist(strings, params_usados['alphabet'], params)
    resultados.append({
        **params,
        'dist': dist,
        'tempo': tempo,
        'history': history
    })
    print(f"Experimento {i+1}/{len(blfga_experimentos)}: dist={dist}, tempo={tempo:.2f}s, params={params}")


Experimento 1/256: dist=135, tempo=439.52s, params={'pop_size': 150, 'initial_blocks': 20, 'min_block_len': 1, 'cross_prob': 0.9, 'mut_prob': 0.5, 'elite_rate': 0.05, 'rediv_freq': 10, 'max_gens': 400, 'max_time': 1200.0, 'seed': 1, 'immigrant_freq': 10, 'immigrant_ratio': 0.2, 'diversity_threshold': 0.4, 'mutation_adapt_N': 10, 'mutation_adapt_factor': 2.0, 'mutation_adapt_duration': 5, 'mutation_type': 'multi', 'mutation_multi_n': 2, 'tournament_k': 2, 'crossover_type': 'one_point', 'niching': False, 'niching_radius': 3, 'refinement_type': 'greedy', 'refine_elites': 'best', 'refine_iter_limit': 100, 'restart_patience': 20, 'restart_ratio': 0.3, 'disable_elitism_gens': 0}
Experimento 2/256: dist=135, tempo=469.94s, params={'pop_size': 150, 'initial_blocks': 20, 'min_block_len': 1, 'cross_prob': 0.9, 'mut_prob': 0.5, 'elite_rate': 0.05, 'rediv_freq': 10, 'max_gens': 400, 'max_time': 1200.0, 'seed': 1, 'immigrant_freq': 10, 'immigrant_ratio': 0.2, 'diversity_threshold': 0.4, 'mutation_a

KeyboardInterrupt: 

In [ ]:

# DataFrame para análise
res_df = pd.DataFrame(resultados)

# Plot comparativo das melhores convergências
plt.figure(figsize=(10,6))
for idx, row in res_df.iterrows():
    plt.plot(row['history'], label=f"{row['pop_size']},{row['cross_prob']},{row['mut_prob']},{row['elite_rate']},{row['max_gens']}")
plt.xlabel('Geração')
plt.ylabel('Distância')
plt.title('Convergência BLF-GA - Variação de Parâmetros')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()

# Exibir top 5 melhores resultados
display(res_df.sort_values('dist').head())